In [1]:
import openai
import tiktoken
import os, json, time
import docx2txt

# to address rate limits with Azure OpenAI API
from tenacity import retry, stop_after_attempt, wait_random_exponential # for exponential backoff

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']

model = os.environ['CHAT_MODEL_NAME']

In [3]:
doc_file_name = "./data/PowerSchoolMicrosoft Architecture Design Session_2023-06-20.docx" 
txt_file_name = './data/final-transcript.txt'

with open(doc_file_name, 'rb') as infile:
    outfile = open(txt_file_name, 'w', encoding='utf-8')
    doc = docx2txt.process(infile)
    lines = doc.splitlines()
    transcript = ""
    for line in lines:
        if '-->' in line:
            continue
        elif line.strip().isdigit():
            transcript += '\n\n'
        else:
            transcript += line.strip() + '\n'  
    
    
    outfile.write(transcript)

    outfile.close()
    infile.close()

In [4]:
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain

llm = AzureChatOpenAI(temperature=0.2,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    deployment_name=model,
    openai_api_key=openai.api_key,
    openai_api_type = openai.api_type,
)

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 5000, chunk_overlap  = 200, length_function = len)
texts = text_splitter.split_text(transcript)
print(f"Number of Chunks: {len(texts)} \n")


Number of Chunks: 12 



In [5]:
## Langchain Version for generating a summary of a long document

# docs = [Document(page_content=t) for t in texts]
# chain = load_summarize_chain(llm, chain_type="map_reduce")
# chain.run(docs)

In [6]:
@retry(wait=wait_random_exponential(min=1, max=30), stop=stop_after_attempt(10))
def get_completion(prompt, model="gpt-35-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature
        max_tokens=150,
    )
    return response.choices[0].message["content"]

In [14]:
summaries = []
for text in texts:
    prompt = f"""
    Summarize the following conversation, delimited by triple backticks, without missing out on any important details. Include people's names as needed based on the context. 
    Conversation: ```{text}```
    """
    
    response = get_completion(prompt)
    summaries.append(response)

In [15]:
summaries[0]

"Hans-Peter Acker asks if everyone is aware of the meeting's purpose and suggests a quick introduction. Arka Chakraborty introduces his team from Power School, including Harshit, Gayatri, Gaurav, and Vishnu. Hans-Peter Acker introduces Catherine from Microsoft and Ellie and Shiva from GPS US. Ellie explains that the meeting is for an architecture design session and asks Power School to share their work. Arka explains that they work on proof of concepts and collaborate with the product team. He shares two examples of their work: question generation and a charting app. They are currently working on integrating these features with Power School's established products."

In [16]:
with open('./data/long_summary.txt', 'w') as file:
    for summary in summaries:
        file.write(summary + '\n')

In [17]:
final_prompt = f"""
Extract and list the key discussion items from the following meeting notes, delimited by triple backticks, without missing out any important details. 
Meeting Notes: ```{" ".join(summaries)}```
"""

response = get_completion(final_prompt)
print(response)

- The meeting is for an architecture design session
- Arka Chakraborty introduces his team from Power School
- Power School works on proof of concepts and collaborates with the product team
- Power School shares two examples of their work: question generation and a charting app
- Power School is currently working on integrating these features with their established products
- Power School discusses their analytics products, including Unified Insights and Connected Intelligence
- Power School is exploring the use of generative AI to create charts through a UI interface
- Power School is working on a Power Tutor app that teaches students and provides evaluation feedback
- The architecture for the app is discussed, with technical difficulties experienced during screen sharing
- The group discusses various metrics and categories for quality checks
- Power School plans to replace GPT 4 with other looms and their AWS bedrock or even open source LMS
- The group discusses security and guardrai

In [18]:
final_prompt = f"""
List out the key discussion items and key takeaways in a structured format from the following meeting notes, delimited by triple backticks. 
Meeting Notes: ```{" ".join(summaries)}```
"""

response = get_completion(final_prompt)
print(response)

Key Discussion Items:
- Introduction of attendees
- Power School's proof of concept work on question generation and charting app integration
- Power School's analytics products, including Unified Insights and Connected Intelligence
- Architecture for Power Tutor app integration with established products
- Technical difficulties with screen sharing
- Quality checks and prompt engineering
- Engagement with each product and deep dive sessions
- Fact-checking and benchmark material
- Design and security measures
- Advising on identified areas and regular meetings
- Development of Power Tutor app using generative AI and Open AI calls
- M Fe micro front end development for embedding tutor within Power School applications
- Legalities and logistics of sharing work

Key Takeaways:
- The meeting was for an architecture design session
- Power School is working on integrating their proof of concept work with established products
- Power School has analytics products that use student and school da

In [13]:
final_prompt = f"""
Provide a comprehensive explanation of the demo made in the following meeting notes, delimited by triple backticks, without missing out on any important details. 
Meeting Notes: ```{" ".join(summaries)}```
"""

response = get_completion(final_prompt)
print(response)

The meeting notes describe a conversation between various teams working on a project related to generative AI and question generation. Arka Chakraborty introduces his team from Power School, including lead engineers and data scientists, and discusses their approach to proof of concepts and gives examples of their work, including question generation and a charting app. They plan to collaborate with the Performance Matters Engineering team to integrate their features. 

The team discusses the architecture for the question generation app, which will also be integrated with an established platform. They are not too concerned about the architecture and are focused on generative AI. There are some technical difficulties with screen sharing and inviting team members to the call. The conversation involves a group discussing a pipeline for generating questions using GPT 4. They are having trouble sharing diagrams and screens, but eventually, Gayathri is able to share a diagram of the pipeline. 